# Import des données

In [1]:
import pymongo

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['myDatabase']

C:\Users\willi\anaconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
import pandas as pd
import numpy as np

json_data_actors = pd.read_json('C:/Users/willi/Desktop/denActors.json', lines=True)

json_data_movies = pd.read_json('C:/Users/willi/Desktop/denMovies.json', lines=True)

json_data_movies['rank'] = pd.to_numeric(json_data_movies['rank'], errors='coerce')

json_data_movies['year'] = pd.to_numeric(json_data_movies['year'], errors='coerce').astype('Int64')
json_data_movies["year"] = json_data_movies["year"].replace({pd.NA:-1})

json_data_directors = pd.read_json('C:/Users/willi/Desktop/denDirectors.json', lines=True)


In [3]:
data_insert_actors = json_data_actors.to_dict(orient='records')
data_insert_movies = json_data_movies.to_dict(orient='records')
data_insert_directors = json_data_directors.to_dict(orient='records')

Insertion des json directors dans la db

In [4]:
db = client['myDatabase']

collection = db['directors']

result = collection.insert_many(data_insert_directors)
print(f"Inserted {len(result.inserted_ids)} documents")

Inserted 86881 documents


Insertion des json movies dans la db

In [5]:
db = client['myDatabase']

collection = db['movies']

result = collection.insert_many(data_insert_movies)
print(f"Inserted {len(result.inserted_ids)} documents")

Inserted 388270 documents


Insert du json actors dans la db:

In [6]:
db = client['myDatabase']

collection = db['actors']

result = collection.insert_many(data_insert_actors)
print(f"Inserted {len(result.inserted_ids)} documents")

Inserted 817719 documents


# Mesure des performances

Query 1

In [124]:
import time
from pymongo import MongoClient

db = client['myDatabase']
collection = db['movies']

nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()
    query1 = {
            "list_genres": "Sci-Fi",
            "rank": {
                "$gt": 8
            }
        }

   
    projection1 = {
        "name": 1,
        "rank": 1,
        "_id": 0 
    }



    
   
    result = collection.find(query1,projection1)

    for document in result:
          pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)
            
temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))


temps_moyen = sum(temps_executions) / len(temps_executions)

print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.11653965711593628 secondes


query 2

In [125]:
import time
from pymongo import MongoClient

db = client['myDatabase']
collection = db['directors']


nb_executions = 10
temps_executions = []
i=0
res = 0
for i in range(nb_executions):
    start_time = time.time()
    query2 = {
            "dict_genres_den_prob.Romance": {
                "$exists": True,
                "$ne": [None, 0]
            }
        }

    projection2 = {
        "first_name": 1,
        "last_name": 1,
        "_id": 0  
    }


    
   
    result = collection.find(query2,projection2)
    for document in result:
        pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)

       
            
temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))

temps_moyen = sum(temps_executions) / len(temps_executions)
print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.0418914258480072 secondes


query 3

In [127]:
import time
from pymongo import MongoClient


db = client['myDatabase']
collection = db['movies']


nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()
    query3 = {
            "list_genres": "Animation",
            "year": 1995
        }
    
    projection3 = {
        "name": 1,
        "_id": 0  
    }



    result = collection.find(query3,projection3)
    
    for document in result:
        pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)


temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))


temps_moyen = sum(temps_executions) / len(temps_executions)

print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.017431825399398804 secondes


query 4

In [128]:
import time
from pymongo import MongoClient


db = client['myDatabase']
collection_directors = db['directors']
collection_movies = db['movies']

nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()
    kurosawa_movies = collection_directors.find_one(
            {
                "first_name": "Akira",
                "last_name": "Kurosawa"
            },
            {
                "list_movies_id": 1,
                "_id": 0
            }
        )

    kurosawa_movies_id = kurosawa_movies["list_movies_id"] if kurosawa_movies else []


    query4 = {
        "id": {
            "$in": kurosawa_movies_id
        },
        "list_genres": "Drama"
    }


    
    
    result = collection_movies.find(query4)
    
    for document in result:
        pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)


temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))


temps_moyen = sum(temps_executions) / len(temps_executions)

print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.15224748849868774 secondes


query 5

In [130]:
import time
from pymongo import MongoClient


db = client['myDatabase']
collection_directors = db['directors']
collection_movies = db['movies']

nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()
    agnes_varda = collection_directors.find_one({
    "first_name": "Agnès",
    "last_name": "Varda"
    })

    id_varda_movies = agnes_varda["list_movies_id"] if agnes_varda else []


    pipeline = [
        {
            "$match": {
                "id": {"$in": id_varda_movies},
                "rank": {
                    '$exists': True,
                    '$ne': float('nan')
                }
            }
        },
        {
            "$group": {
                "_id": None,
                "averageRank": {"$avg": "$rank"}
            }
        }
    ]




    result = collection_movies.aggregate(pipeline)

    
    for document in result:
        pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)


temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))


temps_moyen = sum(temps_executions) / len(temps_executions)

print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.1441022753715515 secondes


query 6

In [134]:
import time
from pymongo import MongoClient

db = client['myDatabase']
collection_directors = db['directors']
collection_movies = db['movies']
nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()

    pipeline = [
        {
            "$project": {
                "first_name": 1,
                "last_name": 1,
                "filteredGenres": {
                    "$filter": {
                        "input": {
                            "$objectToArray": "$dict_genres_den_prob"
                        },
                        "as": "genre",
                        "cond": {
                            "$and": [
                                {"$ne": ["$$genre.v", None]},
                                {"$ne": ["$$genre.v", 0]}
                            ]
                        }
                    }
                }
            }
        },
        {
            "$match": {
                "$and": [
                    {"filteredGenres": {"$exists": True}},
                    {"filteredGenres": {"$ne": None}},
                    {
                        "$expr": {
                            "$eq": [
                                {"$size": "$filteredGenres"},
                                2
                            ]
                        }
                    }
                ]
            }
        },
        {
            "$project": {
                "first_name": 1,
                "last_name": 1
            }
        }
    ]

    result = collection_directors.aggregate(pipeline)

    
    for document in result:
        pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)

temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))
temps_moyen = sum(temps_executions) / len(temps_executions)
print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.21787092089653015 secondes


Query 7

In [135]:
import time
from pymongo import MongoClient


db = client['myDatabase']
collection_directors = db['directors']
collection_movies = db['movies']

nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()

    query = {
        "den_nb_actors": { "$lt": 5 },
        "rank": { "$gt": 7 }
    }


    projection = {
        "name": 1,
        "_id": 0  
    }

 
    result = collection_movies.find(query, projection)

    
    for document in result:
        pass
    
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)


temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))


temps_moyen = sum(temps_executions) / len(temps_executions)

print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.13344335556030273 secondes


Query 8 

In [137]:
import time
from pymongo import MongoClient


db = client['myDatabase']
collection_directors = db['directors']  
collection_movies = db['movies'] 

nb_executions = 10
temps_executions = []
i=0

for i in range(nb_executions):
    start_time = time.time()
    pipeline_directors = [
        {
            "$project": {
                "id": 1,
                "list_movies_id":1,
                "full_name": {
                    "$concat": [ "$first_name", " ", "$last_name" ]
                }
            }
        },
        {
            "$match": {
                "full_name": {
                    "$in": [
                        "Mike Nichols",
                        "Arthur Penn",
                        "William Friedkin",
                        "Stanley Kubrick",
                        "Robert Altman",
                        "Milos Forman",
                        "Martin Scorcese",
                        "Woody Allen",
                        "Michael Cimino",
                        "Francis Ford Coppola"
                    ]
                }
            }
        },
         {
            "$unwind": "$list_movies_id"
        },
        {
            "$group": {
                "_id": None,
                "list_director_id": { "$push": "$list_movies_id" }
            }
        }
    ]
    directors_new_h = collection_directors.aggregate(pipeline_directors)
    directors_new_h_array = list(directors_new_h)
    directors_new_h_id = directors_new_h_array[0]['list_director_id'] 
    pipeline_movies = [
        {
            "$match": {
                "id": {
                    "$in": directors_new_h_id
                }
            }
        },
        {
            "$unwind": "$list_genres"
        },
        {
            "$group": {
                "_id": "$list_genres",
                "bestRank": {
                    "$max": {
                        "$cond": {
                            "if": { "$eq": ["$rank", None] },
                            "then": float("-inf"),
                            "else": "$rank"
                        }
                    }
                },
                "name": { "$first": "$name" }
            }
        }
    ]
    result = collection_movies.aggregate(pipeline_movies)

    for document in result:
        pass
        
    end_time = time.time()
    temps_execution = end_time - start_time
    
    temps_executions.append(temps_execution)
    
temps_executions.remove(max(temps_executions))
temps_executions.remove(min(temps_executions))


temps_moyen = sum(temps_executions) / len(temps_executions)

print(f"Temps d'exécution moyen pour {nb_executions - 2} exécutions : {temps_moyen} secondes")


Temps d'exécution moyen pour 8 exécutions : 0.2217390537261963 secondes
